In [1]:
from copy import deepcopy
import time

class Agente:
    def __init__(self):
        self.percepciones = None
        self.acciones = None
        self.habilitado = True
    
    def programa(self):
        raise Exception("implementar")

class Entorno:
  def __init__(self):
    self.agentes =[]
  
  def get_percepciones (self,a):
    raise Exception("implementar")
  
  def ejecutar (self, a):
    raise Exception("Implementar")
  
  def finalizado (self):
    return not any (a.habilitado for a in self.agentes)
  
  def run(self):
    while not self.finalizado():
      for a in self.agentes:
        print(a, " ")
        self.get_percepciones(a)
        self.ejecutar(a)
  
  def insertar (self,a):
    self.agentes.append(a)


class AgenteBuscador(Agente):
    def __init__(self):
        Agente.__init__(self)
        self.estado_inicial = None
        self.estado_meta = None
        self.funcion_sucesor = []
        self.tecnica = None
        self.nodos_visitados = 0

    def add_funcion(self, f):
        self.funcion_sucesor.append(f)

    def test_objetivo(self, e):
        return e == self.estado_meta

    def generar_hijos(self, e):
        hijos = []
        for fun in self.funcion_sucesor:
            h = fun(e)
            hijos.append(h)
        return hijos

    def get_costo(self, camino):
        raise Exception("Error: No existe implementacion")

    def get_heuristica(self, camino):
        raise Exception("Error: No existe implementacion")

    def get_funcion_a(self, camino):
        return self.get_costo(camino) + self.get_heuristica(camino)

    def mide_tiempo(funcion):
        def funcion_medida(*args, **kwards):
            inicio = time.time()
            c = funcion(*args, **kwards)
            t = time.time() - inicio
            return c, t
        return funcion_medida

    @mide_tiempo
    def programa(self):
        self.frontera = [[self.estado_inicial]]
        visitados = []
        while self.frontera:
            if self.tecnica == "profundidad":
                camino = self.frontera.pop()  
            else:
                camino = self.frontera.pop(0)
            nodo = camino[-1]
            visitados.append(nodo)

            if self.test_objetivo(nodo):
                self.acciones = camino
                break
            else:
                for hijo in self.generar_hijos(nodo):
                    if hijo not in visitados:
                        self.nodos_visitados += 1
                        aux = deepcopy(camino)
                        aux.append(hijo)
                        self.frontera.append(aux)


class AgenteMapu(AgenteBuscador):
    def __init__(self):
        AgenteBuscador.__init__(self)
    
    def es_valida(self, elem, e):
        misioneros, canibales, lado = e
        if lado == 1:
            m_izq, c_izq = misioneros - elem[0], canibales - elem[1]
            m_der, c_der = 3 - m_izq, 3 - c_izq
        else:
            m_der, c_der = misioneros + elem[0], canibales + elem[1]
            m_izq, c_izq = 3 - m_der, 3 - c_der
        
        if m_izq < 0 or c_izq < 0 or m_der < 0 or c_der < 0:
            return False
        if (m_izq > 0 and m_izq < c_izq) or (m_der > 0 and m_der < c_der):
            return False
        return True

    def construir_hijo(self, elem, e):
        misioneros, canibales, lado = e
        if lado == 1:
            nuevo_estado = [misioneros - elem[0], canibales - elem[1], 0]
        else:
            nuevo_estado = [misioneros + elem[0], canibales + elem[1], 1]
        return nuevo_estado

    def generar_hijos(self, e):
        posibilidades = [(1, 0), (2, 0), (0, 1), (0, 2), (1, 1)]
        hijos = [self.construir_hijo(elem, e) for elem in posibilidades if self.es_valida(elem, e)]
        return hijos

In [4]:
import random

def es_valido(estado, agente):
    if(len(agente.generar_hijos(estado)) == 0):
        return False
    return True


def estados_aleatorios(cantidad, agente):
    estados = []
    while len(estados) < cantidad:
        mapus = random.randint(0, 3)
        verdugos = random.randint(0, 3)
        lado = random.randint(0, 1)
        estado = (mapus, verdugos, lado)
        if es_valido(estado, agente):
            estados.append(estado)
    
    return estados

def ejecutar_busqueda(tecnica, estado, agente, complejidad):
        agente.tecnica = tecnica
        agente.estado_inicial = estado
        agente.estado_meta = [0,0,0]
        _, t_amp = agente.programa()
        complejidad[0].append(t_amp)
        complejidad[1].append(agente.nodos_visitados)
        return (len(agente.acciones))

def actualizar_puntos(longitud_profundidad, longitud_amplitud, puntos_profundidad, puntos_amplitud):
    if longitud_profundidad < longitud_amplitud:
        puntos_profundidad += 1
    elif longitud_amplitud < longitud_profundidad:
        puntos_amplitud += 1
    else:
        puntos_profundidad += .5
        puntos_amplitud += .5
    return puntos_profundidad, puntos_amplitud

def calcular_complejidad_algoritmos(agente, cantidad_estados):
    estados = estados_aleatorios(cantidad_estados, agente)
    complejidad_profundidad = [[],[]]
    complejidad_amplitud = [[],[]]
    puntos_profundidad = 0
    puntos_amplitud = 0
    
    for estado in estados:
        longitud_solucion_profundidad = ejecutar_busqueda("profundidad", estado, agente, complejidad_profundidad)
        longitud_solucion_amplitud = ejecutar_busqueda("amplitud", estado, agente, complejidad_amplitud)
        puntos_profundidad, puntos_amplitud = actualizar_puntos(longitud_solucion_profundidad, longitud_solucion_amplitud, puntos_profundidad, puntos_amplitud)
    
    print(f"Probabilidad de que DFS encuentre una solucion optima: {(puntos_profundidad/cantidad_estados)*100}%")
    print(f"Probabilidad de que BFS encuentre una solucion optima: {(puntos_amplitud/cantidad_estados)*100}%")
    complejidad = [(complejidad_amplitud, "amplitud"), (complejidad_profundidad, "profundidad")]
    return complejidad

def calcular_estadisticas(complejidad):
    tiempo_promedio = sum(complejidad[0]) / len(complejidad[0])
    nodos_promedio = sum(complejidad[1]) / len(complejidad[1])
    print(f"Tiempo promedio: {tiempo_promedio:.10f} seg")
    print(f"Nodos visitados promedio: {nodos_promedio:.2f}")

def obtener_estadisticas(complejidades):
    for complejidad in complejidades:
        print(f"Estadisticas para para {complejidad[1]}")
        calcular_estadisticas(complejidad[0])

def ejecutar_pruebas(agente, cantidad_estados):
    complejidad = calcular_complejidad_algoritmos(agente, cantidad_estados)
    obtener_estadisticas(complejidad)


agente = AgenteMapu()
cantidad_estados = 1000
ejecutar_pruebas(agente, cantidad_estados)


Probabilidad de que DFS encuentre una solucion optima: 30.4%
Probabilidad de que BFS encuentre una solucion optima: 69.6%
Estadisticas para para amplitud
Tiempo promedio: 0.0001031518 seg
Nodos visitados promedio: 13328.08
Estadisticas para para profundidad
Tiempo promedio: 0.0000790582 seg
Nodos visitados promedio: 13313.12
